# 🛠️ Setup
---

## This notebook draws from the original paragraph by  ERTUĞRUL DEMIR
- **Notebook**
  - Kaggle URL: https://www.kaggle.com/code/datafan07/train-your-own-tokenizer/notebook

In [2]:
cd /kaggle/input/py-readability-metrics

/kaggle/input/py-readability-metrics


In [3]:
!pip install py_readability_metrics-1.4.5-py3-none-any.whl

Processing ./py_readability_metrics-1.4.5-py3-none-any.whl


In [4]:
cd /kaggle/working

/kaggle/working


In [5]:
import sys
import gc

import pandas as pd
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.metrics import roc_auc_score
import numpy as np
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

from datasets import Dataset
from tqdm.auto import tqdm
from transformers import PreTrainedTokenizerFast

from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier

import seaborn as sns
import matplotlib.pyplot as plt
from readability import Readability
from scipy.sparse import csr_matrix, hstack

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


## Data preparation

In [7]:
test = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/test_essays.csv')
sub = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/sample_submission.csv')
org_train = pd.read_csv('/kaggle/input/llm-detect-ai-generated-text/train_essays.csv')
train = pd.read_csv("/kaggle/input/daigt-v2-train-dataset/train_v2_drcat_02.csv", sep=',')

train = train.drop_duplicates(subset=['text'])
train.reset_index(drop=True, inplace=True)

def extract_scores(text):
    try:
        r = Readability(text)
        scores =  [r.flesch_kincaid().score, r.flesch().score, r.gunning_fog().score, r.coleman_liau().score, r.dale_chall().score, r.ari().score, r.linsear_write().score, r.spache().score]
        return [round(x, 3) + 1000 for x in scores]
    except:
        return [0]*8
    
train = train.sample(1000, random_state = 1)
    
train['scores'] = train['text'].map(extract_scores)
scores_train = train['scores'].apply(pd.Series)
scores_train.columns = [ f"scores_{x}" for x in scores_train.columns]
gc.collect()

test['scores'] = test['text'].map(extract_scores)
scores_test = test['scores'].apply(pd.Series)
scores_test.columns = [ f"scores_{x}" for x in scores_test.columns]
gc.collect()

## Tokenizer

In [8]:
LOWERCASE = False
VOCAB_SIZE = 30522


raw_tokenizer = Tokenizer(models.BPE(unk_token="[UNK]"))
raw_tokenizer.normalizer = normalizers.Sequence([normalizers.NFC()] + [normalizers.Lowercase()] if LOWERCASE else [])
raw_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.BpeTrainer(vocab_size=VOCAB_SIZE, special_tokens=special_tokens)
dataset = Dataset.from_pandas(test[['text']])

def train_corp_iter(): 
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["text"]
        
raw_tokenizer.train_from_iterator(train_corp_iter(), trainer=trainer)
tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)
tokenized_texts_test = []

for text in tqdm(test['text'].tolist()):
    tokenized_texts_test.append(tokenizer.tokenize(text))

tokenized_texts_train = []

for text in tqdm(train['text'].tolist()):
    tokenized_texts_train.append(tokenizer.tokenize(text))

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

## TF-IDF

In [9]:
def dummy(text):
    return text

vectorizer = TfidfVectorizer(ngram_range=(3, 5), lowercase=False, sublinear_tf=True, 
                             analyzer = 'word', tokenizer = dummy,preprocessor = dummy,
                             token_pattern = None, strip_accents='unicode')

vectorizer.fit(tokenized_texts_test)

vocab = vectorizer.vocabulary_

print(vocab)

vectorizer = TfidfVectorizer(ngram_range=(3, 5), lowercase=False, sublinear_tf=True, vocabulary=vocab,
                            analyzer = 'word', tokenizer = dummy, preprocessor = dummy,
                            token_pattern = None, strip_accents='unicode')

tf_train = vectorizer.fit_transform(tokenized_texts_train)
tf_test = vectorizer.transform(tokenized_texts_test)

del vectorizer
gc.collect()

{'ĠAaa Ġbbb Ġccc': 0, 'Ġbbb Ġccc .': 6, 'ĠAaa Ġbbb Ġccc .': 1, 'ĠBbb Ġccc Ġddd': 2, 'Ġccc Ġddd .': 7, 'ĠBbb Ġccc Ġddd .': 3, 'ĠCCC Ġddd Ġeee': 4, 'Ġddd Ġeee .': 8, 'ĠCCC Ġddd Ġeee .': 5}


44

## Classifier ensemble

In [10]:
df_sparse = csr_matrix(scores_train.values)
tf_train = hstack([df_sparse, tf_train])

df_sparse = csr_matrix(scores_test.values)
tf_test = hstack([df_sparse, tf_test])
gc.collect()

y_train = train['label'].values

ValueError: blocks[0,:] has incompatible row dimensions. Got blocks[0,1].shape[0] == 1000, expected 44868.

In [ ]:
if len(test.text.values) <= 5:
    sub.to_csv('submission.csv', index=False)
else:
    clf = MultinomialNB(alpha=0.02)
    sgd_model = SGDClassifier(max_iter=8000, tol=1e-4, loss="modified_huber") 
    p6={'n_iter': 1500,'verbose': -1,'objective': 'cross_entropy','metric': 'auc',
        'learning_rate': 0.05073909898961407, 'colsample_bytree': 0.726023996436955,
        'colsample_bynode': 0.5803681307354022, 'lambda_l1': 8.562963348932286, 
        'lambda_l2': 4.893256185259296, 'min_data_in_leaf': 115, 'max_depth': 23, 'max_bin': 898}
    lgb=LGBMClassifier(**p6)
    cat=CatBoostClassifier(iterations=1000,
                           verbose=0,
                           l2_leaf_reg=6.6591278779517808,
                           learning_rate=0.005689066836106983,
                           allow_const_label=True,loss_function = 'CrossEntropy')
    weights = [0.07,0.31,0.31,0.31]
 
    ensemble = VotingClassifier(estimators=[('mnb',clf),
                                            ('sgd', sgd_model),
                                            ('lgb',lgb), 
                                            ('cat', cat)
                                           ],
                                weights=weights, voting='soft', n_jobs=-1)
    ensemble.fit(tf_train, y_train)
    gc.collect()
    final_preds = ensemble.predict_proba(tf_test)[:,1]
    sub['generated'] = final_preds
    sub.to_csv('submission.csv', index=False)
    sub